In [1]:
import pysolr
from Solr.create_doc import Create_doc
from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems

In [6]:
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":53,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-48"}},\n  "status":"OK"}\n'

In [2]:
def UpadateSolr(path_marc, count, shelf):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = Create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)
        
        count += 1
    return docs, count

In [ ]:
count = 1
e = '1'
p = '1'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs, _ = UpadateSolr(path_marc, count, f"E{e}.P{p}")

In [3]:
count = 1
solr_doc = list()
for e in range(1, 5):
    for p in range(1,6):
        out = f'E{e}/P{p}'
        path_marc = f'Input/{out}/koha.xml'
        docs, count = UpadateSolr(path_marc, count, f"E{e}.P{p}")
        solr_doc = solr_doc+docs




1
Conjecturas e refutações
2
Os donos da paisagem
3
Cooperação Brasil-França
4
<I=     1> Bienal de Pesquisa da Fundação Oswaldo Cruz, 7 a 11 de dezembro de 1998.
5
Contribuição da pós-graduação brasileira para o desenvolvimento sustentável
6
FAPESP
7
Para uma história da FAPESP
8
Relatório de atividades 2003-2008
9
Relatório de atividades 2007
10
Relatório de atividades 2010
11
V Jornada científica de pos-graduação da FIOCRUZ
12
Anais
13
Resumos
14
Anais
15
Resumos
16
Resumos
17
Resumos
18
Resumos
19
Congresso de iniciação científica da Universidade Federal do Amazonas
20
Resumos
21
Resumos
22
Pesquisa científica e tecnológica em saúde
23
Pesquisas recentes em energia, meio ambiente e tecnologia
24
Iniciação à pesquisa científica
25
<1º >Seminário de Ciências da Fiube
26
Encontro sobre o programa de pesquisas para o trópico úmido.
27
A pesquisa no Brasil
28
Resumos do XVII Seminário de Iniciação Científica da UFPA
29
FADESP, Fundação de Amaparo e Desenvolvimento da Pesquisa, Belém-Par

In [10]:
solr.add(solr_doc, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":836}}\n'

In [9]:
solr_doc

[{'id': 1,
  'call': '001 P831c',
  'shelf': 'E1.P1',
  'title': 'Conjecturas e refutações',
  'responsibilities': 'Karl R. Popper; tradução de Sérgio Bath.',
  'place': 'Brasília',
  'publisher': 'Ed. Universidade de Brasília',
  'subject': ['Ciência--Metodologia', 'Teoria do conhecimento'],
  'type': 'Livro',
  'language': 'por',
  'items': ['01-0519'],
  'author': 'Popper, Karl Raimund',
  'serie': 'Pensamento científico ; n. 1',
  'year': '1982',
  'notes': ['Título original: Conjectures and refutations, 4th ed. rev, 1972.']},
 {'id': 2,
  'call': '001.0141 D687',
  'shelf': 'E1.P1',
  'title': 'Os donos da paisagem : estudos sobre divulgação científica',
  'responsibilities': 'Glória Kreinz, Crodowaldo Pavan, organizadores.',
  'place': 'São Paulo',
  'publisher': 'NJR/ECA/USP',
  'subject': ['Divulgação científica'],
  'type': 'Livro',
  'language': 'por',
  'items': ['01-0478'],
  'serie': 'Divulgação Científica ; v. 3',
  'year': '2000'},
 {'id': 3,
  'call': '001.14 C776',
  '

In [1]:
out = f'E1/P1'
path_marc = f'Input/{out}/koha.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')
record = records[0]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)


NameError: name 'parse' is not defined

In [ ]:
doc = Create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)

In [5]:
solr_doc[0]

{'id': 1,
 'call': '001 P831c',
 'shelf': 'E1.P1',
 'title': 'Conjecturas e refutações',
 'responsibilities': 'Karl R. Popper; tradução de Sérgio Bath.',
 'place': 'Brasília',
 'publisher': 'Ed. Universidade de Brasília',
 'subject': ['Ciência--Metodologia', 'Teoria do conhecimento'],
 'type': 'Livro',
 'language': 'por',
 'items': ['01-0519'],
 'author': 'Popper, Karl Raimund',
 'serie': 'Pensamento científico ; n. 1',
 'year': '1982',
 'notes': ['Título original: Conjectures and refutations, 4th ed. rev, 1972.']}